In [17]:
import warnings
warnings.filterwarnings('ignore')
# from datetime import datetime, date
import numpy as np
import pandas as pd
# import yfinance as yf

In [25]:
# A empresa e Indicadores fundamentalistas dados do statusinvest.com.br
#(pode ter atraso, dados não são pegos diretamente no SI)

url = 'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/SI_Acoes.csv'
pd.set_option('display.max_columns', None)
fundsSI = pd.read_csv(url,sep=';' , decimal=',' ,thousands ='.' )

fundsSI =  fundsSI[fundsSI[' LIQUIDEZ MEDIA DIARIA'] > 250000] #Volume diário médio negociado nos últimos 2 meses
fundsSI =  fundsSI[(fundsSI[' LPA'] > 0) & (fundsSI[' LPA'] > 0)]
fundsSI

,TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,EV/EBIT,DIVIDA LIQUIDA / EBIT,DIV. LIQ. / PATRI.,PSR,P/CAP. GIRO,P. AT CIR. LIQ.,LIQ. CORRENTE,ROE,ROA,ROIC,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
1,ABCB4,24.75,6.37,6.05,1.02,0.10,25.50,17.06,15.50,5.50,5.50,NaN,NaN,0.94,-2.86,-0.10,0.05,16.95,1.73,NaN,0.10,0.91,0.11,18.70,17.80,1.662386e+07,24.15,4.09,-0.33,5.864168e+09
2,ABEV3,12.70,5.75,13.27,2.22,1.45,50.30,22.59,18.29,10.74,9.99,-0.75,-0.15,2.43,53.97,-1.99,1.11,16.71,10.94,18.63,0.65,0.34,0.60,10.72,15.72,2.518599e+08,5.73,0.96,0.88,2.000737e+11
5,AESB3,10.40,NaN,39.67,1.40,0.32,35.54,28.78,4.90,6.76,16.72,9.96,2.06,1.94,19.17,-0.38,1.11,3.53,0.80,5.04,0.23,0.72,0.16,NaN,NaN,3.290188e+07,7.43,0.26,-0.59,6.260044e+09
7,AGRO3,23.77,13.51,9.52,1.11,0.70,30.65,20.17,20.86,9.84,11.69,1.85,0.21,1.99,3.40,-1.13,2.14,11.68,7.31,7.59,0.63,0.37,0.35,27.22,15.22,7.614446e+06,21.40,2.50,-0.22,2.440785e+09
13,ALOS3,23.55,2.20,4.31,0.92,0.48,96.37,264.05,140.12,2.29,2.01,-0.28,-0.11,6.04,13.18,-0.54,1.58,21.32,11.20,27.21,0.53,0.43,0.08,25.65,86.64,1.053069e+08,25.62,5.46,0.00,1.351621e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,VULC3,18.13,12.60,8.77,2.49,1.77,40.55,19.67,20.41,9.10,9.12,0.02,0.01,1.79,3.26,-5.91,4.39,28.43,20.16,21.21,0.71,0.29,0.99,14.97,24.47,1.640123e+07,7.27,2.07,0.13,4.949785e+09
610,VVEO3,7.95,NaN,15.86,0.82,0.26,15.90,6.01,1.50,3.95,6.45,2.50,0.52,0.24,0.85,-0.64,2.04,5.17,1.62,9.84,0.31,0.69,1.08,NaN,NaN,1.168530e+07,9.69,0.50,-0.38,2.566424e+09
611,WEGE3,33.20,1.85,28.04,8.64,4.63,31.56,18.80,15.70,23.42,22.93,-0.48,-0.18,4.40,14.49,-14.94,1.86,30.82,16.52,25.44,0.54,0.45,1.05,25.71,34.19,1.815734e+08,3.84,1.18,0.70,1.393510e+11
615,WIZC3,6.26,7.11,10.29,2.18,0.44,58.84,37.07,9.26,2.57,2.84,0.26,0.22,0.95,-17.18,-0.56,0.90,21.18,4.24,22.73,0.20,0.59,0.46,12.37,-9.41,2.804296e+06,2.87,0.61,-0.27,1.001020e+09


In [30]:
fundsSI['valor_intrinseco'] = np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA'])
fundsSI['Delta'] = fundsSI['valor_intrinseco'] / fundsSI['PRECO'] -1
fundsSI["Rank"]   = fundsSI['Delta'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1
fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA']]

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
1,RNEW4,1.32,13.418756,199.0,9.165724,2.47,3.24
2,LUPA3,1.89,8.714987,198.0,3.611104,0.87,3.88
3,MEAL3,1.82,5.982725,197.0,2.287212,0.41,3.88
4,GOAU4,10.00,32.252023,196.0,2.225202,2.66,17.38
5,HBOR3,2.64,8.386596,195.0,2.176741,0.30,10.42
...,...,...,...,...,...,...,...
197,WEGE3,33.20,10.097128,3.0,-0.695870,1.18,3.84
198,RADL3,25.18,6.870426,2.0,-0.727148,0.63,3.33
199,TFCO4,14.30,1.052141,1.0,-0.926424,0.12,0.41
200,GOLL4,2.57,NaN,NaN,NaN,0.03,-6.88


### Porque 22,5?

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é, na verdade, o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce.

Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos.

O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores.

Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.

No próximo tópico, abordaremos a calculadora do preço justo usando a Fórmula de Graham.